# Dump IDs

As I am trying to use the DB less when running on the cluster, I want to work out a quick script to call the NCBI database and dump out ids.

In [1]:
# %load ../start.py
# Load useful extensions

# Activate the autoreload extension for easy reloading of external packages
%reload_ext autoreload
%autoreload 2

# Trun on the water mark
%reload_ext watermark
%watermark -u -d -g

# Load ipycache extension
%reload_ext ipycache
from ipycache import CacheMagics
CacheMagics.cachedir = '../cachedir'

# Add project library to path
import sys
sys.path.insert(0, '../../lib/python')

# The usual suspects
import os
import numpy as np
import pandas as pd

# plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set_context('poster')

# Turn off scientific notation
np.set_printoptions(precision=5, suppress=True)


last updated: 2017-10-12 
Git hash: 16cf23ba57f46d68bd8c213ef07a4c5b3c6993cc


In [2]:
# %load ../../bin/load.py
from pymongo import MongoClient
with open('/home/fearjm/Projects/ncbi_remap/output/.mongodb_host', 'r') as fh:
    host = fh.read().strip()
client = MongoClient(host=host, port=27022)
db = client['sra2']
remap = db['remap']
ncbi = db['ncbi']

In [3]:
df = pd.DataFrame(list(remap.aggregate([
    {'$unwind': '$runs'},
    {
        '$project': {
            '_id': 0,
            'srx': '$srx',
            'srr': '$runs.srr',
            'flag_pre_aln_complete': {
                '$cond': [
                    {
                        '$or': [
                            {'$in': ['complete', '$runs.pre_aln_flags']},
                            {'$in': ['download_bad', '$runs.pre_aln_flags']},
                            {'$in': ['quality_scores_bad', '$runs.pre_aln_flags']},
                            {'$in': ['alignment_bad', '$runs.pre_aln_flags']},
                            {'$in': ['abi_solid', '$runs.pre_aln_flags']},
                        ]
                    },
                    True,
                    False
                ]
            },
        }
        
    },
])))

In [4]:
df2 = pd.DataFrame(list(remap.aggregate([
    {'$unwind': '$runs'},
    {
        '$match': {
            'runs.aln_flags': {'$exists': 1}
        }
    },
    {
        '$project': {
            '_id': 0,
            'srx': '$srx',
            'srr': '$runs.srr',
            'flag_aln_complete': {
                '$cond': [
                    {
                        '$or': [
                            {'$in': ['complete', '$runs.aln_flags']},
                            {'$in': ['alignment_bad', '$runs.aln_flags']},
                        ]
                    },
                    True,
                    False
                ]
            },
        }
        
    },
])))

In [5]:
merged = df.merge(df2, how='left', on=['srx', 'srr'])
merged = merged[['srx', 'srr', 'flag_pre_aln_complete', 'flag_aln_complete']]

In [6]:
df.shape, df2.shape, merged.shape

((35363, 3), (17776, 3), (35363, 4))

In [7]:
merged.isnull().any()

srx                      False
srr                      False
flag_pre_aln_complete    False
flag_aln_complete         True
dtype: bool

In [8]:
merged.fillna(False, inplace=True)

In [9]:
store = pd.HDFStore('../../config/sra.h5')

In [10]:
store.append('ids', merged, data_columns=True, index=False)

In [11]:
store.create_table_index('ids', columns=['srx', 'srr'], optlevel=9, kind='full')

In [12]:
store.root.ids.table

/ids/table (Table(35363,)) ''
  description := {
  "index": Int64Col(shape=(), dflt=0, pos=0),
  "srx": StringCol(itemsize=10, shape=(), dflt=b'', pos=1),
  "srr": StringCol(itemsize=10, shape=(), dflt=b'', pos=2),
  "flag_pre_aln_complete": BoolCol(shape=(), dflt=False, pos=3),
  "flag_aln_complete": BoolCol(shape=(), dflt=False, pos=4)}
  byteorder := 'little'
  chunkshape := (2184,)
  autoindex := True
  colindexes := {
    "srr": Index(9, full, shuffle, zlib(1)).is_csi=True,
    "srx": Index(9, full, shuffle, zlib(1)).is_csi=True}

In [21]:
query = 'flag_pre_aln_complete == False & columns==[srx, srr]'
row = store.select('ids', query)

In [22]:
row

,srx,srr
7,SRX2551010,SRR5244308
8,SRX2551009,SRR5244306
39,SRX2541752,SRR5234226
40,SRX2541751,SRR5234225
41,SRX2541750,SRR5234224
42,SRX2541749,SRR5234223
43,SRX2541748,SRR5234222
44,SRX2541747,SRR5234221
87,SRX062905,SRR206928
88,SRX039370,SRR206929
